In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os

# Setup RAG Index Input Corpus for Past Participant Data (DEPRECATED)
Note that this approach is now deprecated

In [ ]:
os.makedirs(os.path.dirname("data/past_participants_split/"), exist_ok=True)
df = pd.read_csv('data/past_participant_info.csv')
df = df.dropna(subset=['VRF ID', 'Skillset', 'Person Id']).reset_index(drop=True)
train_ratio = 0.8
train_df, eval_df = train_test_split(df['Person Id'], test_size=(1 - train_ratio), random_state=42)
train_df.to_csv("data/past_participants_split/train_ids.csv", index=False)
eval_df.to_csv("data/past_participants_split/eval_ids.csv", index=False)

def create_applicant_info_corpus(applicant_info_csv: str, train_ids_csv: str, max_samples: int = -1) -> str:
    """Create a corpus of applicant information from a CSV file.

    Args:
        applicant_info_csv: Path to the CSV file containing applicant information.
        train_ids_csv: Path to the CSV file containing the train IDs.
        max_samples: Maximum number of samples to use. If -1, use all samples.

    Returns:
        A corpus of applicant information.
    """
    train_ids_df = pd.read_csv(train_ids_csv)
    max_samples = len(train_ids_df) if max_samples == -1 else min(max_samples, len(train_ids_df))
    train_ids_df = train_ids_df.sample(n=max_samples, random_state=42)
    job_assigns_df = pd.read_csv(applicant_info_csv)
    job_assigns_df = train_ids_df.merge(job_assigns_df, on='Person Id', how='inner')
    job_assigns_df['job'] = job_assigns_df['VRF ID'].apply(lambda x: x.split('-')[1])
    job_assigns_df['Skillset'] = job_assigns_df['Skillset'].apply(lambda x: x.replace('\n', ' '))
    job_assigns_df['summary'] = " Participant with skills: " + job_assigns_df['Skillset'] + " was assigned to job: " + job_assigns_df['job'] + ".\n"
    corpus = ''.join(job_assigns_df['summary'])
    return corpus

corpus = create_applicant_info_corpus('data/past_participant_info.csv', 'data/generated_participants_training/train_ids.csv', -1)
with open("data/past_participants_split/train_corpus.txt", "w") as file:
    file.write(corpus)
eval_ids_df = pd.read_csv('data/past_participants_split/eval_ids.csv')
eval_data = eval_ids_df.merge(df, on='Person Id', how='inner')
eval_data["eval_input"] =  " Participant " + eval_data["Person Id"].astype(str) + " has skills: " + eval_data['Skillset']

In [91]:
vrf_df = pd.read_csv("data/vrf_data.csv")
results_df = pd.read_csv("results/results-20241130_160356/eval_results.csv")

In [93]:
def get_depts_from_job(job_title, vrf_df):
    return vrf_df[vrf_df['Job Title'] == job_title]['Department'].drop_duplicates().values

def get_depts_from_job_df(results_df, vrf_df):

    def get_depts(row):
        depts = ""
        for job_title in row[3:]:
            if pd.isna(job_title):
                depts += ",NA, "
            else:
                print(job_title)
                job_depts = get_depts_from_job(job_title, vrf_df)
                print(job_depts)
                depts += job_title + ": " + ", ".join(job_depts) + " "
        return depts

    depts = results_df.apply(get_depts, axis=1)
    return depts

res = get_depts_from_job_df(results_df, vrf_df)
print(res[0])

Data Analyst
['Isha Goushala' 'Rally for Rivers / Cauvery Calling'
 'IYC Communications / Guest care' 'Archives'
 'E-Media Panel / Emedia Operations' 'Farmer Producer Organisation (FPO)'
 'IPC' 'Sadhanapada' 'ILPL - Marketing' 'IT / IT - Infrastructure'
 'Bangalore Temple']
 Data Scientist
[]
 Data Engineer
[]
 Data Architect
[]
 Business Analyst
[]
Data Analyst
['Isha Goushala' 'Rally for Rivers / Cauvery Calling'
 'IYC Communications / Guest care' 'Archives'
 'E-Media Panel / Emedia Operations' 'Farmer Producer Organisation (FPO)'
 'IPC' 'Sadhanapada' 'ILPL - Marketing' 'IT / IT - Infrastructure'
 'Bangalore Temple']
 Data Scientist
[]
 Data Engineer
[]
 Data Architect
[]
 Business Analyst
[]
Mathematics Teacher
[]
 Science Teacher
[]
 Physics Teacher
[]
 Chemistry Teacher
[]
 Biology Teacher
[]
Customer Service Representative
[]
 Client Relations Manager
[]
 Public Relations Specialist
[]
 Community Outreach Coordinator
[]
IT Project Manager
[]
 IT Manager
[]
 IT Director
[]
 IT Spe

In [83]:
vrf_df

,Request Name,Job Title,Job Description,Department,Requester Name,Phone Number,Gender Preference,# of Volunteers,Stage,Skills/Keywords,Add'l Skills,Languages,Work Experience Needed?,Number of Years,Educational Qualification
0,Global Languages Publications - Arabic Transla...,Arabic Translator,1 support coordinator: Assigns tasks to volunt...,Global Languages Publications,Miguel Hernandez,8903816455,Any,4.0,Approved,Soft Skills / Articulate in communication,Basic Computer Skills / Data Entry skills,Arabic / الْعَرَبيّة,False,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Media and Communication / Editor,NaN,NaN,NaN,NaN
2,Global Languages Publications - Translators - ...,Translators,"Bulgarian Translator\tTranslation of text, cre...",Global Languages Publications,Miguel Hernandez,NaN,Any,3.0,Approved,Soft Skills / Articulate in communication,Media and Communication / Editor,Bulgarian / български език,False,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soft Skills / Strong Interpersonal skills (Coo...,NaN,NaN,NaN,NaN
4,Global Languages Publications - French Transla...,French Translator,"French translator\tTranslation of text, creati...",Global Languages Publications,Miguel Hernandez,NaN,Any,4.0,Approved,Soft Skills / Articulate in communication,Arts and Crafts / Video Editor,French / Français,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT / Data Analyst,NaN,NaN,NaN,NaN,NaN
1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Basic Computer Skills / Data Entry skills,NaN,NaN,NaN,NaN,NaN
1334,Ashram Administration - Administrative Activit...,Administrative Activities (Back Office),For Muktesha - Ashram Admin under Swami Vidyav...,Ashram Administration,Niraj Gadakari,NaN,Any,1.0,Draft,Basic Computer Skills / Basic Computer (MS Off...,NaN,NaN,False,NaN,NaN
1335,Hata Yoga School - On ground Support - 2893,On ground Support,- Some one from a medical background. Preferab...,Hata Yoga School,Saravana Raj,9489045358,Any,1.0,Approved,Medical / Allopathy Doctor,NaN,NaN,False,NaN,NaN
